# Notebook to generate templates for GloVe models


In [53]:
import os
import yaml
import numpy as np

from utils import check_folder, read_yaml, save_yaml, write

In [54]:
path_to_main = "/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/code/fMRI/main.py"

In [55]:
subject_dict = {'english': [57, 58, 59, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70,
                    72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 86, 87, 88, 89, 91, 92, 93,
                    94, 95, 96, 97, 98, 99, 100, 101, 103, 104, 105, 106, 108, 109, 110, 113, 114, 115],
                'french':[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20,
                          21, 22, 23, 24, 25, 26, 27, 29, 30
                         ]
               }

In [56]:
hrf_list = [
    'spm', # hrf model used in SPM
    'spm + derivative', # SPM model plus its time derivative (2 regressors)
    'spm + derivative + dispersion', # idem, plus dispersion derivative (3 regressors)
    'glover', # this one corresponds to the Glover hrf
    'glover + derivative', # the Glover hrf + time derivative (2 regressors)
    'glover + derivative + dispersion' # idem + dispersion derivative
]
hrf = 'spm'

In [57]:
language = 'english'

In [58]:
template = {
  # Shared General parameters
  'subject': None,
  'parallel': False,
  'cuda': True,
  'seed': 1111,
  'language': None,
  'path_to_root': "/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/",
  'path_to_fmridata': "/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/data/fMRI",
  'output': "/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/derivatives/fMRI/maps/"  ,
  'input': "/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/data/stimuli-representations/" ,

  # Shared Splitter parameters
  'nb_runs': 9,
  'nb_runs_test': 1,

  # Shared Compression parameters


  # Shared Transformation parameters (includes the making of regressor and scaling)
  'tr': 2.,
  'scaling_mean': True,
  'scaling_var': True,
  'scaling_axis': None,
  'hrf': None,
  'offset_path': "/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/data/onsets-offsets/",
  'duration_path': "/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/",

  # Shared Estimator model parameters
  'base': 10.0,
  'voxel_wise': True,
  'alpha_percentile': 99.9,
  'alpha': None,
  'alpha_min_log_scale': 2,
  'alpha_max_log_scale': 5,
  'nb_alphas': 10,
  'optimizing_criteria': 'R2',
  'estimator_model': 'Ridge()',

  # Maps creation parameters
  'atlas': 'cort-prob-2mm',
  'masker_path': "/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/global_masker_english",
  'smoothed_masker_path': "/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/smoothed_global_masker_english",


  # Models
  'models': None, 
  'model_name': None
}

In [59]:
def write_commands(command_lines, path_to_sh):
    for index, command in enumerate(command_lines):
        write(path_to_sh[index], command)
        queue = 'Nspin_long' # 'Nspin_bigM'
        walltime = '99:00:00'
        output_log = '/home/ap259944/logs/log_o_{}'.format(index)
        error_log = '/home/ap259944/logs/log_e_{}'.format(index)
        job_name = ''.join(os.path.basename(path_to_sh[index]).split('.')[:-1])
        write(job_to_launch_path, f"qsub -q {queue} -N {job_name} -l walltime={walltime} -o {output_log} -e {error_log} {path_to_sh[index]}")


In [60]:
def get_glove_template(
    model_name, 
    embedding_size,
    surname,
    data_compression, 
    ncomponents,
    offset_type,
    duration_type,
    shift_surprisal,
    centering,
    order,
    scaling_type,
    input_template='activations'):
    
    columns_to_retrieve = ['embedding-{}'.format(i) for i in range(1, embedding_size + 1)]
    result = { 
        'model_name': model_name,
        'columns_to_retrieve': str(columns_to_retrieve),
        'surname': surname,
        'data_compression': data_compression,
        'ncomponents': ncomponents,
        'offset_type': offset_type, # word / word+punctuation / ...,
        'duration_type': duration_type,
        'shift_surprisal': shift_surprisal,
        'centering': centering,
        'order': order,
        'scaling_type': scaling_type,
        'input_template': input_template # cls / sep / activations
      }
    return result
    

## Here starts the generation

In [61]:
templates_folder = "/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/tmp_glove/templates/"
sh_folder = "/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/tmp_glove/shell_commands/"
job_to_launch_path = "/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/tmp_glove/jobs.txt"
check_folder(templates_folder)
check_folder(sh_folder)


In [62]:
model_names = ['glove_embeddings'] * 8
embedding_size = [300] * 8
data_compression = [None] * 8
ncomponents = [None] * 8
shift_surprisal = False
centering = [True] * 8
order = ['np.inf', '-np.inf', '0', '1', '-1' ,'2','-2', None]
scaling_type = ['normalize'] * 7 + ['standardize']
names = ['glove_{}_norm_{}_centering'.format(embedding_size[index], order[index]) for index in range(len(order) - 1)] + ['glove_{}_standardize_centering'.format(embedding_size[-1])]
command_lines = []
path_to_sh = []
scaling_axis = 1

In [63]:
template['language'] = language
template['hrf'] = hrf
template['scaling_axis'] = scaling_axis

In [64]:
for index, model_name in enumerate(model_names):
    for subject in subject_dict[language]:
        template['subject'] = subject
        
        # hidden layers comparison
        model = get_glove_template(
                            model_name=model_name, 
                            embedding_size=embedding_size[index],
                            surname=names[index],
                            data_compression=data_compression[index], 
                            ncomponents=ncomponents[index],
                            offset_type="word+punctuation",
                            duration_type=None,
                            shift_surprisal=False,
                            centering=centering[index],
                            order=order[index],
                            scaling_type=scaling_type[index],
                            input_template='activations')

        template['models'] = [model]
        template['model_name'] = names[index] + '_{}'.format(subject)
        yaml_path = os.path.join(templates_folder, names[index] + '_{}.yml'.format(subject))

        save_yaml(template, yaml_path)
        command_lines.append("python {} --yaml_file {}".format(path_to_main, yaml_path))
        path_to_sh.append(os.path.join(sh_folder, names[index] + '_{}.sh'.format(subject)))


In [65]:
write_commands(command_lines, path_to_sh)